In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing

## Model

In [2]:
def join(series):
    #return ' '.join( str(list(series)).strip('[]').split(', ') )
    return ' '.join( map(str, series) )

In [3]:
%run 'metrics.ipynb'
def model_evaluation(actual, prediction):
    return mapk(actual['ad_id'], prediction['ad_id'], k=12)

In [8]:
pd.read_csv("../generated/final/events_clicks_CTR_addocs_dist_train.csv", nrows=100).head()

,display_id,document_id,timestamp,traffic_source,platform,day,hour,geo,ad_id,clicked,adsPerDisplay,clicksPerShows,addoc_id,dist
0,8,1330329,638,1.0,2.0,1,7,2765,95724,0,4,0.034785,1136820,0.020897
1,8,1330329,638,1.0,2.0,1,7,2765,175694,0,4,0.202327,1392479,0.021849
2,8,1330329,638,1.0,2.0,1,7,2765,280430,1,4,0.256853,1601512,0.018556
3,8,1330329,638,1.0,2.0,1,7,2765,329774,0,4,0.052426,1652347,0.025201
4,2657,1271490,185255,1.0,2.0,1,7,2765,70081,0,4,0.150145,933594,0.013171


In [4]:
usecols = ['display_id', 'ad_id', 'clicksPerShows', 'adsPerDisplay', 'platform', 'traffic_source', 'dist', 'clicked']
features = ['clicksPerShows', 'clicksPerShowsM', 'p1', 'p2', 'p3', 't1', 't2', 't3', 'adsPerDisplay', 'dist', 'distM']
#features = ['clicksPerShows', 'clicksPerShowsM']
#features = ['p1', 'p2', 'p3', 't1', 't2', 't3', 'adsPerDisplay']

In [10]:
pd.read_csv("../generated/final/test_index.csv", nrows=10).head()

,index
0,21
1,22
2,23
3,24
4,25


## Preparation

### Step 1

In [11]:
train_nrows, test_nrows = None, None       # 650000, 210000

In [12]:
train_index = pd.read_csv("../generated/final/train_index.csv", nrows=train_nrows)['index'].ravel()
test_index = pd.read_csv("../generated/final/test_index.csv", nrows=test_nrows)['index'].ravel()

In [13]:
nrows = max(train_index[-1], test_index[-1])+1
nrows

87141731

In [14]:
train = pd.read_csv("../generated/final/events_clicks_CTR_addocs_dist_train.csv", usecols = usecols, nrows=nrows)
train.count()

display_id        87141731
traffic_source    87141731
platform          87141731
ad_id             87141731
clicked           87141731
adsPerDisplay     87141731
clicksPerShows    87141731
dist              77941525
dtype: int64

In [15]:
test = train.ix[test_index].copy()
train = train.ix[train_index].copy()

In [16]:
platform_oneshot = pd.DataFrame({'platform':[1,2,3], 'p1':[1,0,0], 'p2':[0,1,0], 'p3':[0,0,1]})
platform_oneshot.head()

,p1,p2,p3,platform
0,1,0,0,1
1,0,1,0,2
2,0,0,1,3


In [17]:
traffic_oneshot = pd.DataFrame({'traffic_source':[1,2,3], 't1':[1,0,0], 't2':[0,1,0], 't3':[0,0,1]})
traffic_oneshot.head()

,t1,t2,t3,traffic_source
0,1,0,0,1
1,0,1,0,2
2,0,0,1,3


In [19]:
train = train.merge(platform_oneshot, on='platform').drop('platform', axis=1)
test = test.merge(platform_oneshot, on='platform').drop('platform', axis=1)

train = train.merge(traffic_oneshot, on='traffic_source').drop('traffic_source', axis=1)
test = test.merge(traffic_oneshot, on='traffic_source').drop('traffic_source', axis=1)

In [21]:
dist_mean = train['dist'].mean()

In [ ]:
train = train.fillna({'dist':dist_mean}, inplace=True)
test = test.fillna({'dist':dist_mean}, inplace=True)

In [25]:
scaler = preprocessing.StandardScaler()
excpt = ['clicksPerShowsM', 'distM'] #, 'p1', 'p2', 'p3', 't1', 't2', 't3', 'adsPerDisplay']
for f in features:
    if not f in excpt:
        print f
        scaler.fit(train[[f]])
        train[f] = scaler.transform(train[[f]]) 
        test[f] = scaler.transform(test[[f]])

clicksPerShows
dist


In [27]:
train.to_csv("../generated/solutions/8_val_train.0.csv", index=False)
test.to_csv("../generated/solutions/8_val_test.0.csv", index=False)

### Step 2

In [5]:
train = pd.read_csv("../generated/solutions/8_val_train.0.csv")

trainM = pd.DataFrame({'clicksPerShowsM':train.groupby('display_id')['clicksPerShows'].min(),
                       'distM':train.groupby('display_id')['dist'].min()
                      }).reset_index()

train = train.merge(trainM, on='display_id')
train.count()

display_id         65355241
ad_id              65355241
clicked            65355241
adsPerDisplay      65355241
clicksPerShows     65355241
dist               65355241
p1                 65355241
p2                 65355241
p3                 65355241
t1                 65355241
t2                 65355241
t3                 65355241
clicksPerShowsM    65355241
distM              65355241
dtype: int64

In [6]:
del trainM

In [7]:
test = pd.read_csv("../generated/solutions/8_val_test.0.csv")
testM = pd.DataFrame({'clicksPerShowsM':test.groupby('display_id')['clicksPerShows'].min(),
                      'distM':test.groupby('display_id')['dist'].min()
                     }).reset_index()
test = test.merge(testM, on='display_id')
test.count()

display_id         21786490
ad_id              21786490
clicked            21786490
adsPerDisplay      21786490
clicksPerShows     21786490
dist               21786490
p1                 21786490
p2                 21786490
p3                 21786490
t1                 21786490
t2                 21786490
t3                 21786490
clicksPerShowsM    21786490
distM              21786490
dtype: int64

In [8]:
del testM

In [9]:
train[features].head()

,clicksPerShows,clicksPerShowsM,p1,p2,p3,t1,t2,t3,adsPerDisplay,dist,distM
0,-1.106421,-1.106421,-0.869339,1.16846,-0.414761,0.589609,-0.386977,-0.382641,-0.943129,-0.337428,-0.453601
1,0.047741,-1.106421,-0.869339,1.16846,-0.414761,0.589609,-0.386977,-0.382641,-0.943129,-0.290188,-0.453601
2,0.423361,-1.106421,-0.869339,1.16846,-0.414761,0.589609,-0.386977,-0.382641,-0.943129,-0.453601,-0.453601
3,-0.984897,-1.106421,-0.869339,1.16846,-0.414761,0.589609,-0.386977,-0.382641,-0.943129,-0.123861,-0.453601
4,-0.311728,-1.004649,-0.869339,1.16846,-0.414761,0.589609,-0.386977,-0.382641,-0.943129,-0.720782,-0.720782


In [70]:
features = ['clicksPerShows', 'clicksPerShowsM', 'dist', 'distM']

## Validation

In [71]:
clf = linear_model.SGDClassifier(random_state=0, loss='log', fit_intercept=True, n_iter=5)
clf.fit(train[features], train['clicked'].ravel())

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False)

In [39]:
clf.classes_

array([0, 1], dtype=int64)

In [26]:
clicks_test_sample = test[['display_id', 'ad_id', 'clicked']]
clicks_test_sample.count()

display_id    21786490
ad_id         21786490
clicked       21786490
dtype: int64

In [27]:
validation = clicks_test_sample[clicks_test_sample['clicked']==1].astype({'ad_id':str}).sort_values('display_id')
validation.count()

display_id    4220025
ad_id         4220025
clicked       4220025
dtype: int64

In [40]:
test[features].head()

,clicksPerShows,dist
0,0.357575,-1.193204
1,-1.292027,-0.610874
2,-1.217672,-0.656453
3,-0.123321,-0.516288
4,-1.186901,-0.118871


In [72]:
test['res'] = pd.Series(clf.predict_proba(test[features])[:,1])
test.head()

,display_id,ad_id,clicked,adsPerDisplay,clicksPerShows,dist,p1,p2,p3,t1,t2,t3,clicksPerShowsM,distM,res
0,19972,120162,1,1.123554,0.357575,-1.193204,-0.869339,1.16846,-0.414761,-1.69604,-0.386977,2.613417,-1.292027,-1.193204,0.250525
1,19972,122279,0,1.123554,-1.292027,-0.610874,-0.869339,1.16846,-0.414761,-1.69604,-0.386977,2.613417,-1.292027,-1.193204,0.063788
2,19972,168342,0,1.123554,-1.217672,-0.656453,-0.869339,1.16846,-0.414761,-1.69604,-0.386977,2.613417,-1.292027,-1.193204,0.068221
3,19972,176585,0,1.123554,-0.123321,-0.516288,-0.869339,1.16846,-0.414761,-1.69604,-0.386977,2.613417,-1.292027,-1.193204,0.172818
4,19972,250080,0,1.123554,-1.186901,-0.118871,-0.869339,1.16846,-0.414761,-1.69604,-0.386977,2.613417,-1.292027,-1.193204,0.069687


In [42]:
clf.coef_, clf.intercept_[0]

(array([[ 0.87560411, -0.01007099]]), -1.6182738461657566)

In [73]:
prediction = test.sort_values(by=['display_id', 'res'], ascending=[True, False])
prediction.count()

display_id         21786490
ad_id              21786490
clicked            21786490
adsPerDisplay      21786490
clicksPerShows     21786490
dist               21786490
p1                 21786490
p2                 21786490
p3                 21786490
t1                 21786490
t2                 21786490
t3                 21786490
clicksPerShowsM    21786490
distM              21786490
res                21786490
dtype: int64

In [74]:
prediction=pd.DataFrame({ 'ad_id': prediction.groupby('display_id')['ad_id'].apply(join) }).reset_index()
prediction.count()

display_id    4220025
ad_id         4220025
dtype: int64

In [75]:
prediction.head()

,display_id,ad_id
0,1,279295 144739 42337 139684 296965 156824
1,7,300808 215967 105766
2,9,151028 19959 140940 104208
3,14,98270 143467 224171 288396
4,26,152193 285992


In [46]:
validation.head()

,display_id,ad_id,clicked
10663775,1,144739,1
6969458,7,105766,1
15168153,9,140940,1
8851096,14,224171,1
7891472,26,152193,1


In [76]:
print 'clicksPerShows :', model_evaluation(validation, prediction)  #0.662869594202      0.588499968978      

clicksPerShows : 0.662827167065


In [36]:
# 0.662870121262
# 0.662827167065

#['clicksPerShows', 'dist']                                   0.662839597671
#['clicksPerShows', 'clicksPerShowsM', 'dist']                0.662822157289
#['clicksPerShows', 'p1', 'p2', 'p3', 'dist']                 0.66283803558
#['clicksPerShows', 'dist', 'distM']                          0.66286541307
#['clicksPerShows', 'clicksPerShowsM', 'dist', 'distM']       0.662827167065